In [23]:
import pandas as pd
from py3dbp import Packer, Bin, Item
import os

#**********************************************#
# Load the Excel file
file_path = 'Optimization Problem.xlsx'
packages_df = pd.read_excel(file_path, sheet_name='Packages')
cartons_df = pd.read_excel(file_path, sheet_name='Cartons')

    
#**********************************************#
if not os.path.isdir("outcome"):         
    os.mkdir("outcome") 
with open("outcome/optimize-solution.csv", "w") as fw:
    fw.write("PackageID,Length,Width,Depth,CartonName,ItemsFit,%Utilization,CartonName,ItemsFit,%Utilization,CartonName,ItemsFit,%Utilization,CartonName,ItemsFit,%Utilization,CartonName,ItemsFit,%Utilization,CartonName,ItemsFit,%Utilization,BestFit\n")   

for index,package in packages_df.iterrows():
    package_id = package['Package_ID']
    length = package['PKG_LNGTH_IN']
    width = package['PKG_WIDTH_IN']
    height = package['PKG_DEPTH_IN']
    item_data = {
        "name": package_id, 
        "length": length, 
        "width": width, 
        "height": height, 
        "weight": 0
    }
    with open('outcome/optimize-solution.csv', 'a') as fw:
        fw.write("{},{},{},{},".format(package_id, round(length,2), round(width,2),round(height,2)))
# Initialize variables to track the best fit
    best_fit_container = None
    best_fit_volume_utilized_percentage = 0
    
    # Calculating the optimization packaging instructions
    for index, carton in cartons_df.iterrows():
        # Define your storage unit as a 'Bin'
        storage_unit = Bin(carton['Description'], carton['ID Length (in)'], carton['ID Width (in)'], carton['ID Height (in)'], 1)
        # Initialize the Packer
        packer = Packer()
        # Add the storage unit to the packer
        packer.add_bin(storage_unit)
        # Add items in batches
        batch_size = 100
        num_batches = 10  # 1000 items in total
        for i in range(num_batches):
            batch_items = [Item(item_data["name"], item_data["length"], item_data["width"], item_data["height"], item_data["weight"]) for _ in range(batch_size)]
            for item in batch_items:
                packer.add_item(item)
        # Run the packing algorithm
        packer.pack()
        # Calculate the volume of the storage unit
        storage_volume = float(storage_unit.width * storage_unit.height * storage_unit.depth)
        # Calculate the volume of a single item
        item_volume = float(item_data["length"] * item_data["width"] * item_data["height"])
        # Calculate the total volume utilized and the number of items fit
        total_items_fit = sum(len(b.items) for b in packer.bins)
        total_volume_utilized = float(total_items_fit * item_volume)
        # Calculate the percentage of volume utilized
        volume_utilized_percentage = (total_volume_utilized / storage_volume) * 100
        # Check if this container is the best fit
        if volume_utilized_percentage > best_fit_volume_utilized_percentage:
            best_fit_container = carton
        des = carton['Description']+"-"+str(carton['ID Total Area (in)'])
        with open('outcome/optimize-solution.csv', 'a') as fw:
            fw.write("{},{},{},".format(des,total_items_fit,round(volume_utilized_percentage,2)))
    with open('outcome/optimize-solution.csv', 'a') as fw: 
        if best_fit_container is None:
            fw.write("{}\n".format('NONE'))
        else:
            fw.write("{}\n".format(best_fit_container["Description"]))
 
 